# 실전 문제 해결 (부족한 데이터셋)

In [1]:
import os
import numpy as np
import tensorflow as tf
import random
from imblearn.over_sampling import BorderlineSMOTE

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## 하이퍼 파라미터

In [2]:
EPOCHS = 100

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비 (불균형한 데이터셋)

In [4]:
cifar10 = tf.keras.datasets.cifar10 # 32x32x3

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# TODO: 학습 데이터를 Imbalanced small dataset으로 변형하기
x_train_small = list()
y_train_small = list()
for x, y in zip(x_train, y_train):
    if (y == 0 and random.randint(0, 100) < 10) or y == 1:
        x_train_small.append(x[:])
        y_train_small.append(y)
        
x_test_small = list()
y_test_small = list()
for x, y in zip(x_test, y_test):
    if y == 0 or y == 1:
        x_test_small.append(x[:])
        y_test_small.append(y)

x_train = np.stack(x_train_small, axis=0)
y_train = np.stack(y_train_small, axis=0)

x_test = np.stack(x_test_small, axis=0)
y_test = np.stack(y_test_small, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습 (불균형한 데이터셋)

In [5]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
172/172 [==============================] - 7s 18ms/step - loss: 0.4331 - accuracy: 0.8983 - precision: 0.9211 - recall: 0.9716 - val_loss: 0.8591 - val_accuracy: 0.6095 - val_precision: 0.5625 - val_recall: 0.9850
Epoch 2/100
172/172 [==============================] - 4s 25ms/step - loss: 0.2454 - accuracy: 0.9146 - precision: 0.9293 - recall: 0.9808 - val_loss: 0.6751 - val_accuracy: 0.6770 - val_precision: 0.6102 - val_recall: 0.9800
Epoch 3/100
172/172 [==============================] - 3s 17ms/step - loss: 0.2255 - accuracy: 0.9202 - precision: 0.9348 - recall: 0.9808 - val_loss: 0.5111 - val_accuracy: 0.7665 - val_precision: 0.6944 - val_recall: 0.9520
Epoch 4/100
172/172 [==============================] - 4s 26ms/step - loss: 0.2094 - accuracy: 0.9275 - precision: 0.9410 - recall: 0.9820 - val_loss: 0.8536 - val_accuracy: 0.6240 - val_precision: 0.5716 - val_recall: 0.9900
Epoch 5/100
172/172 [==============================] - 3s 16ms/step - loss: 0.2018 - accuracy: 0

172/172 [==============================] - 4s 25ms/step - loss: 0.0763 - accuracy: 0.9705 - precision: 0.9790 - recall: 0.9888 - val_loss: 0.6855 - val_accuracy: 0.8010 - val_precision: 0.7210 - val_recall: 0.9820
Epoch 38/100
172/172 [==============================] - 3s 17ms/step - loss: 0.0729 - accuracy: 0.9710 - precision: 0.9794 - recall: 0.9890 - val_loss: 0.9308 - val_accuracy: 0.7515 - val_precision: 0.6710 - val_recall: 0.9870
Epoch 39/100
172/172 [==============================] - 3s 17ms/step - loss: 0.0644 - accuracy: 0.9745 - precision: 0.9831 - recall: 0.9890 - val_loss: 0.7051 - val_accuracy: 0.8030 - val_precision: 0.7258 - val_recall: 0.9740
Epoch 40/100
172/172 [==============================] - 4s 25ms/step - loss: 0.0680 - accuracy: 0.9745 - precision: 0.9818 - recall: 0.9904 - val_loss: 0.5083 - val_accuracy: 0.8455 - val_precision: 0.7877 - val_recall: 0.9460
Epoch 41/100
172/172 [==============================] - 3s 17ms/step - loss: 0.0659 - accuracy: 0.9761 - 

172/172 [==============================] - 3s 17ms/step - loss: 0.0238 - accuracy: 0.9925 - precision: 0.9948 - recall: 0.9970 - val_loss: 1.0243 - val_accuracy: 0.8190 - val_precision: 0.7420 - val_recall: 0.9780
Epoch 74/100
172/172 [==============================] - 3s 17ms/step - loss: 0.0168 - accuracy: 0.9951 - precision: 0.9966 - recall: 0.9980 - val_loss: 1.2218 - val_accuracy: 0.8065 - val_precision: 0.7269 - val_recall: 0.9820
Epoch 75/100
172/172 [==============================] - 4s 17ms/step - loss: 0.0218 - accuracy: 0.9918 - precision: 0.9944 - recall: 0.9966 - val_loss: 1.2536 - val_accuracy: 0.7965 - val_precision: 0.7156 - val_recall: 0.9840
Epoch 76/100
172/172 [==============================] - 3s 17ms/step - loss: 0.0225 - accuracy: 0.9923 - precision: 0.9954 - recall: 0.9962 - val_loss: 0.9966 - val_accuracy: 0.8260 - val_precision: 0.7515 - val_recall: 0.9740
Epoch 77/100
172/172 [==============================] - 4s 25ms/step - loss: 0.0165 - accuracy: 0.9938 - 

## 데이터셋 준비 (BorderlineSMOTE)

In [6]:
# TODO: BorderlineSMOTE 적용하기
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])).astype(np.float32)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3])).astype(np.float32)

smote = BorderlineSMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습 (BorderlineSMOTE)

In [7]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
313/313 [==============================] - 8s 22ms/step - loss: 0.6470 - accuracy: 0.7251 - precision: 0.7328 - recall: 0.7086 - val_loss: 0.4999 - val_accuracy: 0.7780 - val_precision: 0.7439 - val_recall: 0.8480
Epoch 2/100
313/313 [==============================] - 6s 18ms/step - loss: 0.4201 - accuracy: 0.8211 - precision: 0.8423 - recall: 0.7902 - val_loss: 0.4316 - val_accuracy: 0.8055 - val_precision: 0.7755 - val_recall: 0.8600
Epoch 3/100
313/313 [==============================] - 7s 22ms/step - loss: 0.3474 - accuracy: 0.8609 - precision: 0.8913 - recall: 0.8220 - val_loss: 0.5543 - val_accuracy: 0.7700 - val_precision: 0.7009 - val_recall: 0.9420
Epoch 4/100
313/313 [==============================] - 5s 17ms/step - loss: 0.2766 - accuracy: 0.8940 - precision: 0.9251 - recall: 0.8574 - val_loss: 0.3761 - val_accuracy: 0.8350 - val_precision: 0.8526 - val_recall: 0.8100
Epoch 5/100
313/313 [==============================] - 7s 22ms/step - loss: 0.2252 - accuracy: 0

313/313 [==============================] - 5s 17ms/step - loss: 0.0184 - accuracy: 0.9943 - precision: 0.9956 - recall: 0.9930 - val_loss: 0.9285 - val_accuracy: 0.8405 - val_precision: 0.7826 - val_recall: 0.9430
Epoch 38/100
313/313 [==============================] - 7s 22ms/step - loss: 0.0237 - accuracy: 0.9924 - precision: 0.9952 - recall: 0.9896 - val_loss: 0.8921 - val_accuracy: 0.8460 - val_precision: 0.7898 - val_recall: 0.9430
Epoch 39/100
313/313 [==============================] - 5s 16ms/step - loss: 0.0230 - accuracy: 0.9930 - precision: 0.9946 - recall: 0.9914 - val_loss: 1.2021 - val_accuracy: 0.8095 - val_precision: 0.7336 - val_recall: 0.9720
Epoch 40/100
313/313 [==============================] - 5s 17ms/step - loss: 0.0226 - accuracy: 0.9926 - precision: 0.9940 - recall: 0.9912 - val_loss: 1.0970 - val_accuracy: 0.8165 - val_precision: 0.7514 - val_recall: 0.9460
Epoch 41/100
313/313 [==============================] - 7s 21ms/step - loss: 0.0302 - accuracy: 0.9896 - 

313/313 [==============================] - 7s 21ms/step - loss: 0.0048 - accuracy: 0.9990 - precision: 0.9994 - recall: 0.9986 - val_loss: 1.7939 - val_accuracy: 0.8035 - val_precision: 0.7260 - val_recall: 0.9750
Epoch 74/100
313/313 [==============================] - 5s 17ms/step - loss: 0.0158 - accuracy: 0.9952 - precision: 0.9958 - recall: 0.9946 - val_loss: 1.6723 - val_accuracy: 0.8175 - val_precision: 0.7422 - val_recall: 0.9730
Epoch 75/100
313/313 [==============================] - 7s 21ms/step - loss: 7.6129e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.6918 - val_accuracy: 0.8240 - val_precision: 0.7500 - val_recall: 0.9720
Epoch 76/100
313/313 [==============================] - 5s 17ms/step - loss: 5.2949e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.7199 - val_accuracy: 0.8195 - val_precision: 0.7437 - val_recall: 0.9750
Epoch 77/100
313/313 [==============================] - 5s 16ms/step - loss: 4.1078e-04 - accurac